In [18]:
import os
import random
import geopandas as gpd
import h3
import pandas as pd
from shapely.geometry import Point, Polygon

In [19]:
class H3MockDataGenerator:
    def __init__(self, shapefile_path, csv_path, output_path, num_points=9999, resolution=9):
        self.shapefile_path = shapefile_path
        self.csv_path = csv_path
        self.output_path = output_path
        self.num_points = num_points
        self.resolution = resolution
        self.polygon = self.load_shapefile().geometry.unary_union

    def load_shapefile(self):
        gdf = gpd.read_file(self.shapefile_path)
        return gdf.to_crs("EPSG:4326")

    def generate_random_location(self):
        minx, miny, maxx, maxy = self.polygon.bounds
        while True:
            lat = random.uniform(miny, maxy)
            lon = random.uniform(minx, maxx)
            point = Point(lon, lat)
            if self.polygon.contains(point):
                return lat, lon

    def generate_mock_data(self):
        data = [{
            "hex9": h3.geo_to_h3(*self.generate_random_location(), self.resolution),
            "value": random.randint(1, 9999)
        } for _ in range(self.num_points)]
        return pd.DataFrame(data)

    def save_to_csv(self, df):
        if not os.path.exists('hex'):
            os.makedirs('hex')
        df.to_csv(self.csv_path, index=False)

    def convert_h3_indices_to_geometries(self, df):
        geometries = [Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in df["hex9"]]
        return gpd.GeoDataFrame(df, geometry=geometries)

    def run(self):
        df = self.generate_mock_data()
        self.save_to_csv(df)
        df = pd.read_csv(self.csv_path)
        gdf = self.convert_h3_indices_to_geometries(df)
        gdf.to_file(self.output_path)

In [20]:
if __name__ == "__main__":
    generator = H3MockDataGenerator(
        shapefile_path="../../shapefiles/zh_poly.shp",
        csv_path="../../hex/h3_farm_mock_data.csv",
        output_path="../../app_data/h3_indices_7.shp"
    )
    generator.run()